# 文本预处理
---
文本预处理是自然语言处理（NLP）任务中一个非常重要的步骤，旨在将原始的文本数据转换为可以被机器学习算法或深度学习模型理解和处理的格式。预处理的目标是去除不必要的信息、提取有用的特征，并降低数据的噪音。

## 词元化（Tokenization）
词元化是将原始文本拆分成一个个独立的“词元”（Token），词元可以是单词、子词、字符甚至标点符号。词元化的目标是将文本表示为计算机可以理解的基本单元。词元化是NLP的第一步，因为模型只能处理结构化的数据，而原始文本通常是一个未经分隔的字符串。
**常见的词元化方法：**

1. 基于空格的词元化：这是最简单的词元化方法，按空格分割。适用于英文等西方语言。

- 例如，"I am happy" -> ["I", "am", "happy"]

2. 正则表达式分词：使用正则表达式定义分词规则，可以灵活地处理标点符号、特殊字符等。

- 例如，"Hello, world!" -> ["Hello", "world"]

3. 基于规则的中文分词：中文的分词更复杂，因为中文词语之间没有明确的分隔符。通常使用基于词典的分词算法，如Jieba。

- 例如，"我喜欢自然语言处理" -> ["我", "喜欢", "自然语言处理"]

4. 子词分割（Subword Tokenization）：针对长词或复杂词，使用BPE（Byte Pair Encoding）或WordPiece等方法将词分割成更小的子词或字符。子词分割在处理未知词汇（OOV）时非常有效。

- 例如，使用BPE处理"unbelievable" -> ["un", "##believe", "##able"]

5. 字符级分词：将句子按字符分割，适用于某些细粒度的任务。

- 例如，"Hello" -> ["H", "e", "l", "l", "o"]

## 词表（Vocabulary）
词表是一个包含所有词元的集合，用于将词元映射到唯一的索引（ID）或数值表示。在构建词表时，通常会对所有词元进行计数和排序，然后选取一定数量的高频词元，组成最终的词表。

**词表的作用：**

词表定义了模型可以识别的所有词元，所有不在词表中的词元都会被标记为`<UNK>`（未知词），或者通过子词方式分割处理。
词表用于将词元转换为数值索引，这是模型理解和处理文本的基础。

**构建词表的步骤：**

1. 词频统计：遍历所有文档，统计每个词元的频率。
2. 设定词表大小：根据统计结果选择前N个最常见的词元，组成词表。通常，词表的大小是一个超参数。
3. 添加特殊标记：在词表中添加特殊的词元，如`<PAD>`（用于补全）、`<START>`（句子开始）、`<END>`（句子结束）、`<UNK>`（未知词）等。

## 词元化与词表的关系
词元化和词表是互相配合的步骤。词元化负责将文本拆分为独立的词元，而词表则将这些词元转换成模型可以理解的数值或向量。一个典型的处理流程如下：

1. 对文本进行词元化，将句子切分成词元序列。
2. 根据词表，将每个词元映射为对应的ID。
3. 得到一个由词元ID构成的序列，用于模型输入。

## 简单代码实现


In [ ]:
import collections
import re
from d2l import torch as d2l

In [ ]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

In [ ]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

In [ ]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [ ]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

In [ ]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])